In [1]:
from core.utils import Tibanna
from core import ff_utils
from uuid import UUID
import ast

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

schema_name = {}
profiles = ff_utils.get_metadata('/profiles/', connection=ff, frame = 'raw')
for key, value in profiles.iteritems():
    schema_name[key] = value['id'].split('/')[-1][:-5]

def is_uuid(value):
    #md5 qualifies as uuid
    if '-' not in value:
        return False
    try:
        UUID(value, version=4)
        return True
    except:
        return False

def find_uuids(val):
    vals = []
    if not val:
        return []
    elif isinstance(val, basestring):
        if is_uuid(val):
            vals = [val]
        else:
            return []
    else:
        text =str(val)
        text_list = [i for i in text. split("'") if len(i) == 36]
        vals = [i for i in text_list if is_uuid(i)]
    return vals
    

In [2]:
import json
import copy
def record_object(uuid, store, item_uuids):
    #keep list of fields that only exist in frame embedded (revlinks) that you want connected
    
    add_from_embedded = {'file_fastq':['workflow_run_inputs', 'workflow_run_outputs'],
                         'file_processed':['workflow_run_inputs', 'workflow_run_outputs']
                        }
#     add_from_embedded = {}
  
    #find schema name, store as obj_key, create empty list if missing in store

    obj_type = ff_utils.get_metadata(uuid, connection=ff)['@type'][0]

    try:
        obj_key = schema_name[obj_type]
    except:
        #print 'CAN NOT FIND', obj_type, uuid
        return store, item_uuids
    if obj_key not in store:
        store[obj_key] = []

    add_uuids = []
    resp = ff_utils.get_metadata(uuid, connection=ff, frame = 'raw')
    
    if resp['status'] not in ['submission in progress', 'current', 'released', 'in review by lab', 'uploaded']:
        print obj_key, uuid, resp['status']
        
    # add raw frame to store and uuid to list
    if uuid not in item_uuids:
        store[obj_key].append(resp)
        item_uuids.append(uuid)
    # this case should not happen actually
    else:
        return store, item_uuids
    
    fields_to_check = copy.deepcopy(resp)
  
    # check if any field from the embedded frame is required
    add_fields = add_from_embedded.get(obj_key)
    if add_fields:
        emb_resp = ff_utils.get_metadata(uuid, connection=ff, frame = 'embedded')
        for a_field in add_fields:
            field_val = emb_resp.get(a_field)
            if field_val:
                fields_to_check[a_field] = field_val
        
    for key, value in fields_to_check.iteritems():
        # uuid formatted fields to skip
        if key == 'uuid':
            continue
        uuid_in_val = find_uuids(value)
        
        for a_uuid in uuid_in_val:
            if a_uuid not in item_uuids:
                store, item_uuids = record_object(a_uuid, store, item_uuids)   
    return store, item_uuids
  



#only run with a single experiment set for the rest of the code to run smoothly
my_exp_set = dekker_dil[1]

print my_exp_set
store_dict = {} 
item_uuids = []
store, uuids = record_object(my_exp_set, store_dict, item_uuids)
print(len(uuids))
print(len(list(set(uuids))))
    


# for a_type in store:
#     filename = 'Jsons/' + a_type + '.json'
#     with open(filename, 'w') as outfile:
#         json.dump(store[a_type], outfile, indent=4)




03a396f5-7379-4f86-a26b-232b5e9c7853
workflow 3feedadc-50f9-4bb4-919b-09a8b731d0cc released to project
software 42adeb1b-c46c-4ba8-b191-56036ddcd544 released to project
software 58ed98d7-10d8-4c51-8166-4a813c62ef8c released to project
file_reference 4a6d10ee-2edb-4402-a98f-0edb1d58f5e9 released to project
software 9baf9f73-2d59-4eca-a326-340007b2a8b8 released to project
software b43e5d6d-f1a6-4713-9b48-8f5c060dad6c released to project
software b0c823f7-ca1e-44fb-b0f6-3eefb19eae46 released to project
software c1c772a8-5d1c-4481-8ab7-30729f3f6cb2 released to project
software f3907ef2-a9fe-4324-bc54-253a4cd53e3c released to project
workflow d3f25cd3-e726-4b3c-a022-48f844474b41 released to project
workflow 2324ad76-ff37-4157-8bcc-3ce72b7dace9 released to project
file_reference 595763c6-58d3-4ec4-8f04-3dbb88ed4736 released to project
software 15731f58-868f-44ab-b4ed-77b82ecd032b released to project
90
90


In [7]:
## This part will only work with a single experiment set stored in "store"
## my_exp_set is the defined experiment set
my_exp_set = my_exp
Procs = [[i['accession'],i['source_experiments']] for i in store['file_processed']]
patches={}
for p_file, exp in Procs:
    if len(exp) == 1:
        if patches.get(exp[0]):
            patches[exp[0]].append(p_file)
        else:
            patches[exp[0]] = [p_file,]
            
    elif len(exp) == 2:
        if patches.get(my_exp_set):
            patches[my_exp_set].append(p_file)
        else:
            patches[my_exp_set] = [p_file,]
print patches

{u'3860d5a0-17ac-4ca2-a3af-dcd1bb614105': [u'4DNFISGG3QI2', u'4DNFIVMFGFPM', u'4DNFIJSALZWE'], u'63ca31d3-7e02-4bae-99bc-4adb584e299d': [u'4DNFID3TYFDP', u'4DNFIF3WVA21', u'4DNFI3G7ACRU'], '03a396f5-7379-4f86-a26b-232b5e9c7853': [u'4DNFIGDS8FEX', u'4DNFID68JQY9', u'4DNFIJYULXT7', u'4DNFIE2YNQZZ']}


In [9]:
for a_key in patches.keys():
            patch_data = {'processed_files': patches[a_key]}
            ff_utils.patch_metadata(patch_data, obj_id=a_key ,connection=ff)
            print 'done'


done
done
done


In [ ]:
track_loop_sets = [
'9b1063f5-c891-44a8-b44a-3b4879257366',
'c5db8085-92bd-4d06-a22c-7958e4a78e19',]

dekker_mc = [
'51d9ac15-e55e-4f64-85e5-01998e349cf1',
'998573fb-384e-4792-a621-254d705d4ec9',
]

gianluca = [
'8e9cfe23-aac5-4e66-9514-e880fe34cbc4',
'e4510669-23bb-45cb-a6a0-f6bb0cf75dd0',
'1c75ed00-ba21-4484-b510-f0875d814bc7',
'3ebd99e8-db53-44e9-a831-3e4813d2dd99',
'906bd763-6439-4445-9e4a-5f814b246070',
]

kaufman_sets = [
'c85de504-96cf-4856-b7f3-398e6bf1e37b',
'c50006d4-a98d-4366-bd5b-650fee1696ee',
'43586c61-a527-43d3-b95a-1bc063fa1fb7',
'e2b336fb-dda9-4eff-9125-0603cc99916e',
'f41f9b4b-9f13-4e25-abed-13fcf99b5a73',
'1ed849f5-cf49-4190-84a9-161f13906c21',
'c33e4a8d-cd5e-4bf2-8973-88a46efd2173',
'd5fd5f93-8ffc-4bdc-8db9-1457d119248b',
]

dekker_dil = ['233740be-ffbe-4acc-b6a4-24bbee96ca73',
              '03a396f5-7379-4f86-a26b-232b5e9c7853'

]

sprite_sets = ['cc82a2af-95e7-4447-b571-87d4785b33ec']